In [ ]:
import tables
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.fft import fft, fftfreq
from scipy.signal import welch
%matplotlib inline

cmap = {'PF': 'k', 'pan': 'r'}
width, height = 5, 2
method = 'PSD'

In [ ]:
files = {'PF': '../IEEE39_PowerFactory_5.000_4.330_4.470_3.570_4.330_4.350_3.770_3.470_3.450_4.200.h5',
        'pan': '../IEEE39_pan_5.000_4.330_4.470_3.570_4.330_4.350_3.770_3.470_3.450_4.200.h5'}
fids = {k: tables.open_file(v) for k,v in files.items()}
params = {key: fid.root.parameters.read() for key,fid in fids.items()}
time = {key: fid.root.time.read() / 60 for key,fid in fids.items()}
stoch_load = {'pan': np.squeeze(fids['pan'].root.OU.read()),
              'PF': np.squeeze(fids['PF'].root.noise_bus_3.read())}
buses = 3, 14, 17
lines = (3, 4), (14, 15), (16, 17), (1, 39)
omega = {}
Pe, Qe = {}, {}
Vd, Vq = {}, {}
Id, Iq = {}, {}

omega_ref = np.squeeze(fids['pan'].root.omega_ref.read())
delta_ref = np.squeeze(fids['PF'].root.delta_ref.read())
phi = np.squeeze(fids['PF'].root.phi.read()) / np.pi * 180
f_out = np.squeeze(fids['PF'].root.f_out.read())
V_exc = np.squeeze(fids['PF'].root.V_exc.read())

for key,fid in fids.items():
    omega[key] = {}
    Pe[key], Qe[key] = {}, {}
    Vd[key], Vq[key] = {}, {}
    Id[key], Iq[key] = {}, {}
    for bus in buses:
        omega[key][bus] = np.squeeze(fid.root[f'omegael_bus{bus}'].read())
        Vd[key][bus] = np.squeeze(fid.root[f'Vd_bus{bus}'].read())
        Vq[key][bus] = np.squeeze(fid.root[f'Vq_bus{bus}'].read())
        if key == 'pan':
            omega[key][bus] += omega_ref - 1
    for bus1,bus2 in lines:
        if bus1 not in Pe:
            Pe[key][bus1] = {}
            Qe[key][bus1] = {}
            Id[key][bus1] = {}
            Iq[key][bus1] = {}
        Pe[key][bus1][bus2] = np.squeeze(fid.root[f'Pe_line_{bus1}_{bus2}'].read())
        Qe[key][bus1][bus2] = np.squeeze(fid.root[f'Qe_line_{bus1}_{bus2}'].read())
        Id[key][bus1][bus2] = np.squeeze(fid.root[f'Id_line_{bus1}_{bus2}'].read())
        Iq[key][bus1][bus2] = np.squeeze(fid.root[f'Iq_line_{bus1}_{bus2}'].read())
for fid in fids.values():
    fid.close()

In [ ]:
if np.abs(stoch_load['pan'] - stoch_load['PF']).max() < 1e-10:
    print('The stochastic loads are the same.')
else:
    raise Exception('The stochastic loads are not the same.')

## Bus angular frequencies

### Time series

In [ ]:
coeff = {'PF': 1/params['PF']['F0'][0], 'pan': 1}
rows = len(buses) + 1
cols = 1
fig,ax = plt.subplots(rows, cols, figsize=(cols*width, rows*height), sharex=True)
xlim = [0, 60]
for i,bus in enumerate(buses):
    for key in 'pan','PF':
        ax[i].plot(time[key], omega[key][bus] * coeff[key], color=cmap[key], lw=1)
    ax[i].set_xlim(xlim)
    ax[i].set_ylabel(r'$\omega_{{{}}}$ [p.u.]'.format(bus))
ax[-1].plot(time['PF'], stoch_load['PF'], 'k', lw=1)
ax[-1].set_xlabel('Time [min]')
ax[-1].set_ylabel(r'$\Delta$P [MW]')
for a in ax:
    for side in 'right','top':
        a.spines[side].set_visible(False)
fig.tight_layout()

### Distributions and spectra

In [ ]:
N_samples = time['PF'].size
dt = {k: np.diff(v[:2])[0] for k,v in time.items()}
Fs = {k: 1/dt[k] for k in dt}
freq = {k: fftfreq(N_samples, dt[k])[:N_samples//2] for k in dt}

coeff = {'PF': 1/params['PF']['F0'][0], 'pan': 1}
rows = len(buses) + 1
cols = 2
fig,ax = plt.subplots(rows, cols, figsize=(cols*width/2, rows*height))
for i,bus in enumerate(buses):
    for key in 'pan','PF':
        x = omega[key][bus] * coeff[key]
        n,edges = np.histogram(x, bins=51, density=True)
        ax[i,0].plot(edges[:-1], n, color=cmap[key], lw=1)
        if method == 'FFT':
            xf = 2.0 / N_samples * np.abs(fft(x)[:N_samples // 2])
            ax[i,1].semilogx(freq[key], 20 * np.log10(xf), color=cmap[key], lw=1)
        elif method == 'PSD':
            f, Pxx = welch(x, Fs[key], nperseg=1024)
            ax[i,1].loglog(f, Pxx, color=cmap[key], lw=1)
        ax[i,0].set_xlabel('$\omega_{{{}}}$ [p.u.]'.format(bus))
        ax[i,1].set_xlabel('Freq [Hz]')
        for side in 'right','top':
            for j in range(cols):
                ax[i,j].spines[side].set_visible(False)
x = stoch_load['PF']
n,edges = np.histogram(x, bins=51, density=True)
ax[-1,0].plot(edges[:-1], n, color=cmap[key], lw=1)
if method == 'FFT':
    xf = 2.0 / N_samples * np.abs(fft(x)[:N_samples // 2])
    ax[-1,1].semilogx(freq[key], 20 * np.log10(xf), color=cmap[key], lw=1)
elif method == 'PSD':
    f, Pxx = welch(x, Fs[key], nperseg=1024)
    ax[-1,1].loglog(f, Pxx, color=cmap[key], lw=1)
for side in 'right','top':
    for j in range(cols):
        ax[-1,j].spines[side].set_visible(False)
ax[-1,0].set_xlabel(r'$\Delta$P [MW]')
ax[-1,1].set_xlabel('Freq [Hz]')
for a in ax:
    a[0].set_ylabel('PDF')
    a[1].set_ylabel(method)
fig.tight_layout()

### Correlation with the stochastic load time series

In [ ]:
coeff = {'PF': 1/params['PF']['F0'][0], 'pan': 1}
bus = 3
y = stoch_load['PF']
fig,ax = plt.subplots(1, 2, figsize=(8,4), sharex=True, sharey=True)
for a,sim in zip(ax, ('PF','pan')):
    x = omega[sim][bus] * coeff[sim]
    r, _ = pearsonr(x, y)
    a.plot(x, y, cmap[sim] + '.', markersize=1)
    dx = x.max() - x.min()
    dy = y.max() - y.min()
    y_pos = y.max() - dy/10 if r > 0 else y.min() + dy/10
    a.text(x.min() + dx/50, y_pos, r'$r^2 = {:.4f}$'.format(r**2))
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.set_title(sim if sim == 'PF' else sim.capitalize())
    a.set_xlabel(r'$\omega_{{{}}}$'.format(bus))
ax[0].set_ylabel(r'$\Delta$P [MW]')
fig.tight_layout()

### Time series correlation between Pan and PowerFactory

In [ ]:
coeff = {'PF': 1/params['PF']['F0'][0], 'pan': 1}
rows = len(buses)
cols = 1
fig,ax = plt.subplots(rows, cols, figsize=(cols*width, rows*height))
for i,bus in enumerate(buses):
    x, y = omega['pan'][bus] * coeff['pan'], omega['PF'][bus] * coeff['PF']
    r, p = pearsonr(x, y)
    ax[i].plot(x, y, 'k.', markersize=1)
    dx = x.max() - x.min()
    dy = y.max() - y.min()
    y_pos = y.max() - dy/10 if r > 0 else y.min() + dy/10
    ax[i].text(x.min() + dx/50, y_pos, r'$r^2 = {:.4f}$'.format(r**2))
    for side in 'right', 'top':
        ax[i].spines[side].set_visible(False)
    ax[i].set_title('Bus {}'.format(bus))
for a in ax:
    a.set_ylabel('PowerFactory')
ax[-1].set_xlabel('Pan')
fig.tight_layout()

## Active and reactive powers

### Time series

In [ ]:
coeff = {'PF': 1, 'pan': 1e-6}
rows = len(lines)
cols = 2
fig,ax = plt.subplots(rows, cols, figsize=(cols*width, rows*height))
for i,(bus1,bus2) in enumerate(lines):
    for key in 'pan','PF':
        ax[i,0].plot(time[key], np.abs(Pe[key][bus1][bus2]) * coeff[key], color=cmap[key], lw=1)
        ax[i,1].plot(time[key], np.abs(Qe[key][bus1][bus2]) * coeff[key], color=cmap[key], lw=1)
        for side in 'right','top':
            ax[i,0].spines[side].set_visible(False)
            ax[i,1].spines[side].set_visible(False)
        ax[i,0].set_ylabel('Pe$_{{{}-{}}}$ [MW]'.format(bus1, bus2))
        ax[i,1].set_ylabel('Qe$_{{{}-{}}}$ [MVAR]'.format(bus1, bus2))
for i in range(2):
    ax[-1,i].set_xlabel('Time [min]')
fig.tight_layout()

### Distributions and spectra

In [ ]:
N_samples = time['PF'].size
dt = {k: np.diff(v[:2])[0] for k,v in time.items()}
Fs = {k: 1/dt[k] for k in dt}
freq = {k: fftfreq(N_samples, dt[k])[:N_samples//2] for k in dt}

coeff = {'PF': 1, 'pan': 1e-6}
rows = len(lines)
cols = 4
fig,ax = plt.subplots(rows, cols, figsize=(cols*width/2, rows*height))
for i,(bus1,bus2) in enumerate(lines):
    for key in 'pan','PF':
        x = np.abs(Pe[key][bus1][bus2]) * coeff[key]
        n,edges = np.histogram(x, bins=51, density=True)
        ax[i,0].plot(edges[:-1], n, color=cmap[key], lw=1)
        if method == 'FFT':
            xf = 2.0 / N_samples * np.abs(fft(x)[:N_samples // 2])
            ax[i,1].loglog(freq[key], 20 * np.log10(xf), color=cmap[key], lw=1)
        elif method == 'PSD':
            f, Pxx = welch(x, Fs[key], nperseg=1024)
            ax[i,1].loglog(f, Pxx, color=cmap[key], lw=1)
        x = np.abs(Qe[key][bus1][bus2]) * coeff[key]
        n,edges = np.histogram(x, bins=51, density=True)
        ax[i,2].plot(edges[:-1], n, color=cmap[key], lw=1)
        if method == 'FFT':
            xf = 2.0 / N_samples * np.abs(fft(x)[:N_samples // 2])
            ax[i,3].loglog(freq[key], 20 * np.log10(xf), color=cmap[key], lw=1)
        elif method == 'PSD':
            f, Pxx = welch(x, Fs[key], nperseg=1024)
            ax[i,3].loglog(f, Pxx, color=cmap[key], lw=1)
        for side in 'right','top':
            for j in range(cols):
                ax[i,j].spines[side].set_visible(False)
        ax[i,0].set_xlabel('Pe$_{{{}-{}}}$ [MW]'.format(bus1, bus2))
        ax[i,1].set_xlabel('Freq [Hz]')
        ax[i,2].set_xlabel('Qe$_{{{}-{}}}$ [MVAR]'.format(bus1, bus2))
        ax[i,3].set_xlabel('Freq [Hz]')
for a in ax:
    a[0].set_ylabel('PDF')
    a[2].set_ylabel('PDF')
    a[1].set_ylabel(method)
    a[3].set_ylabel(method)
fig.tight_layout()

### Time series correlation between Pan and PowerFactory

In [ ]:
coeff = {'PF': 1, 'pan': 1e-6}
rows = len(lines)
cols = 2
fig,ax = plt.subplots(rows, cols, figsize=(cols*width, rows*height))
for i,(bus1,bus2) in enumerate(lines):
    for j,var in enumerate((Pe, Qe)):
        x, y = var['pan'][bus1][bus2] * coeff['pan'], var['PF'][bus1][bus2] * coeff['PF']
        r, p = pearsonr(x, y)
        ax[i,j].plot(x, y, 'k.', markersize=1)
        dx = x.max() - x.min()
        dy = y.max() - y.min()
        y_pos = y.max() - dy/10 if r > 0 else y.min() + dy/10
        ax[i,j].text(x.min() + dx/50, y_pos, r'$r^2 = {:.4f}$'.format(r**2))
        for side in 'right', 'top':
            ax[i,j].spines[side].set_visible(False)
        ax[i,j].set_title('Line {} - {}'.format(bus1, bus2))
for a in ax[:,0]:
    a.set_ylabel('PowerFactory')
for a in ax[-1,:]:
    a.set_xlabel('Pan')
fig.tight_layout()

## Bus voltages

### Time series of d and q components

In [ ]:
coeff = {'PF': 1, 'pan': 1e-3}
rows = len(buses)
cols = 2
fig,ax = plt.subplots(rows, cols, figsize=(cols*width, rows*height), sharex=True)
for i,bus in enumerate(buses):
    for key in 'pan','PF':
        ax[i,0].plot(time[key], np.abs(Vd[key][bus]) * coeff[key], color=cmap[key], lw=1)
        ax[i,1].plot(time[key], np.abs(Vq[key][bus]) * coeff[key], color=cmap[key], lw=1)
        for side in 'right','top':
            ax[i,0].spines[side].set_visible(False)
            ax[i,1].spines[side].set_visible(False)
        ax[i,0].set_ylabel('Vd$_{{{}}}$ [kV]'.format(bus))
        ax[i,1].set_ylabel('Vq$_{{{}}}$ [kV]'.format(bus))
for i in range(2):
    ax[-1,i].set_xlabel('Time [min]')
fig.tight_layout()

### Time series of magnitude and phase

In [ ]:
coeff = {'PF': 1, 'pan': 1e-3}
rows = len(buses)
cols = 2
fig,ax = plt.subplots(rows, cols, figsize=(cols*width, rows*height), sharex=True)
for i,bus in enumerate(buses):
    for key in 'pan','PF':
        mag = np.sqrt((Vd[key][bus] * coeff[key]) ** 2 + (Vq[key][bus] * coeff[key]) ** 2)
        phi = np.arctan2(Vq[key][bus], Vd[key][bus])
        ax[i,0].plot(time[key], mag, color=cmap[key], lw=1)
        ax[i,1].plot(time[key], phi, color=cmap[key], lw=1)
        for side in 'right','top':
            ax[i,0].spines[side].set_visible(False)
            ax[i,1].spines[side].set_visible(False)
        ax[i,0].set_ylabel(r'V$_{{{}}}$ [kV]'.format(bus))
        ax[i,1].set_ylabel(r'Phase V$_{{{}}}$ [rad]'.format(bus))
for i in range(2):
    ax[-1,i].set_xlabel('Time [min]')
# ax[0,0].set_xlim([0,10])
fig.tight_layout()

### Distributions and spectra

In [ ]:
N_samples = time['PF'].size
dt = {k: np.diff(v[:2])[0] for k,v in time.items()}
Fs = {k: 1/dt[k] for k in dt}
freq = {k: fftfreq(N_samples, dt[k])[:N_samples//2] for k in dt}

coeff = {'PF': 1, 'pan': 1e-3}
rows = len(buses)
cols = 4
fig,ax = plt.subplots(rows, cols, figsize=(cols*width/2, rows*height))
for i,bus in enumerate(buses):
    for key in 'pan','PF':
        x = np.abs(Vd[key][bus]) * coeff[key]
        n,edges = np.histogram(x, bins=51, density=True)
        ax[i,0].plot(edges[:-1], n, color=cmap[key], lw=1)
        if method == 'FFT':
            xf = 2.0 / N_samples * np.abs(fft(x)[:N_samples // 2])
            ax[i,1].loglog(freq[key], 20 * np.log10(xf), color=cmap[key], lw=1)
        elif method == 'PSD':
            f, Pxx = welch(x, Fs[key], nperseg=1024)
            ax[i,1].loglog(f, Pxx, color=cmap[key], lw=1)
        x = np.abs(Vq[key][bus]) * coeff[key]
        n,edges = np.histogram(x, bins=51, density=True)
        ax[i,2].plot(edges[:-1], n, color=cmap[key], lw=1)
        if method == 'FFT':
            xf = 2.0 / N_samples * np.abs(fft(x)[:N_samples // 2])
            ax[i,3].loglog(freq[key], 20 * np.log10(xf), color=cmap[key], lw=1)
        elif method == 'PSD':
            f, Pxx = welch(x, Fs[key], nperseg=1024)
            ax[i,3].loglog(f, Pxx, color=cmap[key], lw=1)
        for side in 'right','top':
            for j in range(cols):
                ax[i,j].spines[side].set_visible(False)
        ax[i,0].set_xlabel('Vd$_{{{}}}$ [kV]'.format(bus))
        ax[i,1].set_xlabel('Freq [Hz]')
        ax[i,2].set_xlabel('Vq$_{{{}}}$ [kV]'.format(bus))
        ax[i,3].set_xlabel('Freq [Hz]')
for a in ax:
    a[0].set_ylabel('PDF')
    a[2].set_ylabel('PDF')
    a[1].set_ylabel(method)
    a[3].set_ylabel(method)
fig.tight_layout()

### Time series correlation between Pan and PowerFactory

In [ ]:
coeff = {'PF': 1, 'pan': 1e-3}
rows = len(buses)
cols = 2
fig,ax = plt.subplots(rows, cols, figsize=(cols*width, rows*height))
for i,bus in enumerate(buses):
    for j,var in enumerate((Vd, Vq)):
        x, y = var['pan'][bus] * coeff['pan'], var['PF'][bus] * coeff['PF']
        r, p = pearsonr(x, y)
        ax[i,j].plot(x, y, 'k.', markersize=1)
        dx = x.max() - x.min()
        dy = y.max() - y.min()
        y_pos = y.max() - dy/10 if r > 0 else y.min() + dy/10
        ax[i,j].text(x.min() + dx/50, y_pos, r'$r^2 = {:.4f}$'.format(r**2))
        for side in 'right', 'top':
            ax[i,j].spines[side].set_visible(False)
        ax[i,j].set_title('Bus {}'.format(bus))
for a in ax[:,0]:
    a.set_ylabel('PowerFactory')
for a in ax[-1,:]:
    a.set_xlabel('Pan')
fig.tight_layout()

## Line currents

### Time series of d and q components

In [ ]:
coeff = {'PF': 1e3, 'pan': 1 / np.sqrt(3)}
rows = len(lines)
cols = 2
fig,ax = plt.subplots(rows, cols, figsize=(cols*width, rows*height), sharex=True)
for i,(bus1,bus2) in enumerate(lines):
    for key in 'pan','PF':
        ax[i,0].plot(time[key], Id[key][bus1][bus2] * coeff[key], color=cmap[key], lw=1)
        ax[i,1].plot(time[key], Iq[key][bus1][bus2] * coeff[key], color=cmap[key], lw=1)
        for side in 'right','top':
            ax[i,0].spines[side].set_visible(False)
            ax[i,1].spines[side].set_visible(False)
        ax[i,0].set_ylabel('Id$_{{{}-{}}}$ [A]'.format(bus1, bus2))
        ax[i,1].set_ylabel('Iq$_{{{}-{}}}$ [A]'.format(bus1, bus2))
for i in range(2):
    ax[-1,i].set_xlabel('Time [min]')
fig.tight_layout()

### Time series of magnitude and phase

In [ ]:
coeff = {'PF': 1e3, 'pan': 1 / np.sqrt(3)}
rows = len(lines)
cols = 2
fig,ax = plt.subplots(rows, cols, figsize=(cols*width, rows*height), sharex=True)
for i,(bus1,bus2) in enumerate(lines):
    for key in 'pan','PF':
        ire = Id[key][bus1][bus2] * coeff[key]
        iim = Iq[key][bus1][bus2] * coeff[key]
        mag = np.sqrt(ire ** 2 + iim ** 2)
        phi = np.arctan2(iim, ire)
        ax[i,0].plot(time[key], mag, color=cmap[key], lw=1)
        ax[i,1].plot(time[key], phi, color=cmap[key], lw=1)
        for side in 'right','top':
            ax[i,0].spines[side].set_visible(False)
            ax[i,1].spines[side].set_visible(False)
        ax[i,0].set_ylabel(r'I$_{{{}-{}}}$ [A]'.format(bus1, bus2))
        ax[i,1].set_ylabel(r'Phase I$_{{{}-{}}}$ [rad]'.format(bus1, bus2))
for i in range(2):
    ax[-1,i].set_xlabel('Time [min]')
# ax[0,0].set_xlim([0,10])
fig.tight_layout()

### Distributions and spectra

In [ ]:
N_samples = time['PF'].size
dt = {k: np.diff(v[:2])[0] for k,v in time.items()}
Fs = {k: 1/dt[k] for k in dt}
freq = {k: fftfreq(N_samples, dt[k])[:N_samples//2] for k in dt}

coeff = {'PF': 1e3, 'pan': 1 / np.sqrt(3)}
rows = len(lines)
cols = 4
fig,ax = plt.subplots(rows, cols, figsize=(cols*width/2, rows*height))
for i,(bus1,bus2) in enumerate(lines):
    for key in 'pan','PF':
        x = np.abs(Id[key][bus1][bus2]) * coeff[key]
        n,edges = np.histogram(x, bins=51, density=True)
        ax[i,0].plot(edges[:-1], n, color=cmap[key], lw=1)
        if method == 'FFT':
            xf = 2.0 / N_samples * np.abs(fft(x)[:N_samples // 2])
            ax[i,1].loglog(freq[key], 20 * np.log10(xf), color=cmap[key], lw=1)
        elif method == 'PSD':
            f, Pxx = welch(x, Fs[key], nperseg=1024)
            ax[i,1].loglog(f, Pxx, color=cmap[key], lw=1)
        x = np.abs(Iq[key][bus1][bus2]) * coeff[key]
        n,edges = np.histogram(x, bins=51, density=True)
        ax[i,2].plot(edges[:-1], n, color=cmap[key], lw=1)
        if method == 'FFT':
            xf = 2.0 / N_samples * np.abs(fft(x)[:N_samples // 2])
            ax[i,3].loglog(freq[key], 20 * np.log10(xf), color=cmap[key], lw=1)
        elif method == 'PSD':
            f, Pxx = welch(x, Fs[key], nperseg=1024)
            ax[i,3].loglog(f, Pxx, color=cmap[key], lw=1)
        for side in 'right','top':
            for j in range(cols):
                ax[i,j].spines[side].set_visible(False)
        ax[i,0].set_xlabel(r'Id$_{{{}-{}}}$ [A]'.format(bus1, bus2))
        ax[i,1].set_xlabel('Freq [Hz]')
        ax[i,2].set_xlabel('Iq$_{{{}-{}}}$ [kV]'.format(bus1, bus2))
        ax[i,3].set_xlabel('Freq [Hz]')
for a in ax:
    a[0].set_ylabel('PDF')
    a[2].set_ylabel('PDF')
    a[1].set_ylabel(method)
    a[3].set_ylabel(method)
fig.tight_layout()

### Time series correlation between Pan and PowerFactory

In [ ]:
coeff = {'PF': 1e3, 'pan': 1 / np.sqrt(3)}
rows = len(lines)
cols = 2
fig,ax = plt.subplots(rows, cols, figsize=(cols*width, rows*height))
for i,(bus1,bus2) in enumerate(lines):
    for j,var in enumerate((Id, Iq)):
        x, y = var['pan'][bus1][bus2] * coeff['pan'], var['PF'][bus1][bus2] * coeff['PF']
        r, p = pearsonr(x, y)
        ax[i,j].plot(x, y, 'k.', markersize=1)
        dx = x.max() - x.min()
        dy = y.max() - y.min()
        y_pos = y.max() - dy/10 if r > 0 else y.min() + dy/10
        ax[i,j].text(x.min() + dx/50, y_pos, r'$r^2 = {:.4f}$'.format(r**2))
        for side in 'right', 'top':
            ax[i,j].spines[side].set_visible(False)
        ax[i,j].set_title('Line {}-{}'.format(bus1, bus2))
for a in ax[:,0]:
    a.set_ylabel('PowerFactory')
for a in ax[-1,:]:
    a.set_xlabel('Pan')
fig.tight_layout()